### List Comprehension은 메모리를 누수하지 않는다.

python2에서 list comprehension 변수의 범위는 list comprehension 내부에 한정되는 게 아니라 list comprehension이 정의된 함수 전체에 해당한다. 따라서 list comprehension을 정의하고 내부에서 쓰이는 변수를 comprehension 외부에서 접근하는 것이 가능하다. comprehension이 종료되어도 바깥에서 변수를 접근하는게 가능해서 메모리 누수라고 하는 것 같다. 

In [1]:
x = 'abc'
dummy = [ord(x) for x in x]
print(dummy)

[97, 98, 99]


아래 예제를 보면 python2에서 정의된 `dummy`에는 `['s']`가 저장되어 있는데, list comprehension 내부에 정의된 `x`가 고유의 범위를 갖는 변수가 아니라
for문을 돌면서 매번 값이 갱신되는 지역변수로 간주되기 때문이다.

In [1]:
# ➜  ~ python
Python 2.7.10 (default, Oct  6 2017, 22:29:07)
[GCC 4.2.1 Compatible Apple LLVM 9.0.0 (clang-900.0.31)] on darwin
Type "help", "copyright", "credits" or "license" for more information.
>>> x = 'my precious'
>>> dummy = [spell for spell in x]
>>> print x
>>> 's'
>>> dummy
['s']
>>> exit()
➜  ~ python3
Python 3.6.4 (v3.6.4:d48ecebad5, Dec 18 2017, 21:07:28)
[GCC 4.2.1 (Apple Inc. build 5666) (dot 3)] on darwin
Type "help", "copyright", "credits" or "license" for more information.
>>> x = 'my precious'
>>> dummy = [spell for spell in x]
>>> print(spell)
>>> NameError: name 'spell' is not defined
>>> dummy
['m', 'y', ' ', 'p', 'r', 'e', 'c', 'i', 'o', 'u', 's']

SyntaxError: invalid syntax (<ipython-input-1-7c45a3439f20>, line 2)

python3 에서는 `x`변수가 list comprehension 내부에서 고유한 범위를 가진다.

### list comprehension이 함수를 리턴한다면 

In [11]:
def r(i):
    return lambda x: x + i
funcs = []
for i in range(0, 10):
    funcs.append(r(i))

print('nested function is used.')
for i in range(0, 10):
    print(funcs[i](1), end=' ')
    
funcs_li = [lambda x: x + i for i in range(0, 10)]
print('\nlist comprehension used.')
print('i:', i)
for i in range(0, 10):
    print(funcs_li[i](1), end=' ')

nested function is used.
1 2 3 4 5 6 7 8 9 10 
list comprehension used.
i: 9
10 10 10 10 10 10 10 10 10 10 

1. 리스트 컴프리헨션 생성.(for-loop을 돌면서 i는 9로 갱신됨.)
2. 생성된 리스트를 순회하면서 함수를 호출.
3. 리스트 컴프리헨션은 람다 함수의 클로저이므로 람다 함수를 호출하는 시기에 i의 값은 9. <br>
   (클로저 변수는 nested function이 호출되기 전까지 evaluate되지 않는다.)
4. 따라서 `funcs_li` 내부의 모든 람다 함수는 10을 리턴한다.

위와 같이 의도하지 않은 현상이 생긴다면 `functools.partial()`을 이용하자. 
> The partial() is used for partial function application which “freezes” some portion of a function’s arguments and/or keywords resulting in a new object with a simplified signature. https://docs.python.org/2/library/functools.html

출처 : https://stackoverflow.com/questions/1107210/python-lambda-problems

In [15]:
import functools
funcs_li = []
for i in range(0, 10):
    funcs_li.append(functools.partial(lambda x: x + i))
print('partial is used.')
for i in range(0, 10):
    print(funcs_li[i](1), end=' ')

partial is used.
1 2 3 4 5 6 7 8 9 10 

### comrehensions

리스트 뿐만 아니라 딕셔너리와 집합 또한 comprehensions으로 만들 수 있다. <br>
출처 : https://medium.freecodecamp.org/python-list-comprehensions-vs-generator-expressions-cef70ccb49db

In [3]:
dict_comp = {x:chr(65+x) for x in range(1, 11)}

In [5]:
print(dict_comp)
print(type(dict_comp))

{1: 'B', 2: 'C', 3: 'D', 4: 'E', 5: 'F', 6: 'G', 7: 'H', 8: 'I', 9: 'J', 10: 'K'}
<class 'dict'>


In [6]:
set_comp = { x ** 3 for x in range(1, 11) if x % 2 == 0}

In [8]:
print(set_comp)
print(type(set_comp))

{512, 64, 8, 1000, 216}
<class 'set'>


제너레이터 표현식은 튜플을 사용한다.

In [54]:
genexp = (x for x in ['A' * 1000])
import sys
print(sys.getsizeof(genexp))
print(sys.getsizeof(['A' * 1000]))

120
72


In [17]:
next(genexp)

'hi'

In [18]:
next(genexp)

'안녕'

`map()`과 `filter()`를 list comprehension 으로 대체할 수 있다.

In [26]:
symbols = 'aBcDeFg'
beyond_ascii = [ord(s) for s in symbols if ord(s) > 65] 

In [27]:
beyond_ascii

[97, 66, 99, 68, 101, 70, 103]

In [30]:
beyond_ascii = list(filter(lambda c: c > 65, map(ord, symbols)))

In [31]:
beyond_ascii

[97, 66, 99, 68, 101, 70, 103]

list comprehension으로 두 개 이상의 반복 가능한 자료형의 데카르트 곱을 구현할 수 있다.

In [32]:
colors = ['black', 'white']

In [33]:
sizes = ['S', 'M', 'L']

In [35]:
tshirts = [(color, size) for color in colors 
                           for size in sizes]
tshirts

[('black', 'S'),
 ('black', 'M'),
 ('black', 'L'),
 ('white', 'S'),
 ('white', 'M'),
 ('white', 'L')]

In [37]:
tshirts = [(color, size) for size in sizes 
                           for color in colors]
tshirts  # 반복의 순서를 변경하려면 for문의 순서를 변경한다.

[('black', 'S'),
 ('white', 'S'),
 ('black', 'M'),
 ('white', 'M'),
 ('black', 'L'),
 ('white', 'L')]

제너레이터 표현식은 한 번에 한 항목을 생성하므로 n개의 항목을 미리 한번에 정의하는 리스트와 달리 메모리 소모를 피할 수 있다.

In [49]:
for tshirt in (f'{c}, {s}' for c in colors for s in sizes):
    print(tshirt)

black, S
black, M
black, L
white, S
white, M
white, L


## array

```python
class array.array(typecode[, initializer])
```
<br>

> A new array whose items are restricted by typecode, and initialized from the optional initializer value, which must be a list, a bytes-like object, or iterable over elements of the appropriate type.
If given a list or string, the initializer is passed to the new array’s fromlist(), frombytes(), or fromunicode() method (see below) to add initial items to the array. Otherwise, the iterable initializer is passed to the extend() method.
https://docs.python.org/3.4/library/array.html#array.array


In [40]:
import array
a = array.array('I', [ord(symbol) for symbol in symbols])  
# list comprehension의 결과를 엘리먼트로 갖는 unsigned int 타입의 배열 생성

In [42]:
a.itemsize

4

In [43]:
a.pop()

103

In [45]:
a.typecode

'I'

In [47]:
array.typecodes  # available type code in array

'bBuhHiIlLqQfd'

## tuple

튜플은 레코드를 담고있다. 튜플의 각 항목은 레코드의 필드 하나를 의미하며 항목의 위치가 의미를 결정한다. <br>
튜플을 필드의 집합으로 사용하는 경우에는 항목 수가 고정되어 있고 항목의 순서가 중요하다.

In [2]:
city, zipcode, address, phone = (
    '화성시',
    '18484',
    '안알랴줌',
    '01012341234'
)

언패킹은 튜플에만 한정되는 게 아니라 반복가능한 객체라면 모두 적용할 수 있다. iterable unpacking : https://www.python.org/dev/peps/pep-3132/

In [59]:
t = divmod(20, 8)
print(t)
quotient, remainder = divmod(*t)
print(quotient)
print(remainder)

(2, 4)
0
2


`divmod(a, b)`
> 두 개의 (복소수가 아닌) 숫자를 인자로 취하고 정수 나누기를 사용할 때의 몫과 나머지로 구성된 한 쌍의 숫자를 돌려줍니다. 두 인자의 형이 다른 경우, 이 항 산술 연산자에 대한 규칙이 적용됩니다. 정수의 경우, 결과는 (a // b, a % b) 와 같습니다. 

`_`(underscore)와 같은 더미 변수를 placeholder로 사용해서 관심 없는 부분은 언패킹할 때 무시할 수 있다.

In [60]:
import os
_, filename = os.path.split('/home/daeun.kim/test.py')
print(filename)

test.py


튜플은 언패킹할 때 일부 항목에만 관심이 있는 경우에는 `*`을 사용할 수도 있다.(어느 위치에서나 사용 가능.)

In [62]:
print(range(5))

range(0, 5)


In [63]:
a, b, *rest = range(0,5)

In [64]:
a, b, rest

(0, 1, [2, 3, 4])

In [65]:
a, b, *rest = range(0, 3)

In [66]:
a, b, rest

(0, 1, [2])

In [68]:
a, b, *rest = range(0, 2)

In [69]:
a, b, rest

(0, 1, [])

In [70]:
*a, b, rest = range(0, 5)

In [71]:
a, b, rest

([0, 1, 2], 3, 4)

In [72]:
*a, b, rest = range(0, 4)

In [73]:
a, b, rest

([0, 1], 2, 3)

**unpacking nested tuple**

In [74]:
metro_areas = [
    ('Tokyo', 'JP', 36.933, (35.689722, 139.691667)),   
    ('Delhi NCR', 'IN', 21.935, (28.613889, 77.208889)),
    ('Mexico City', 'MX', 20.142, (19.433333, -99.133333)),
    ('New York-Newark', 'US', 20.104, (40.808611, -74.020386)),
    ('Sao Paulo', 'BR', 19.649, (-23.547778, -46.635833)),
]

In [75]:
for name, cc, pop, (lat, lon) in metro_areas:
    print(f'{name} | {lat} | {lon}')

Tokyo | 35.689722 | 139.691667
Delhi NCR | 28.613889 | 77.208889
Mexico City | 19.433333 | -99.133333
New York-Newark | 40.808611 | -74.020386
Sao Paulo | -23.547778 | -46.635833


**named tuple**

필드에 이름을 붙일 필요가 있을 때는 `namedtuple()`을 사용한다.

```python
collections.namedtuple(typename, field_names, *, rename=False, defaults=None, module=None)
```
> Returns a new tuple subclass named typename. The new subclass is used to create tuple-like objects that have fields accessible by attribute lookup as well as being indexable and iterable. Instances of the subclass also have a helpful docstring (with typename and field_names) and a helpful __repr__() method which lists the tuple contents in a name=value format.

> The field_names are a sequence of strings such as ['x', 'y']. Alternatively, field_names can be a single string with each fieldname separated by whitespace and/or commas, for example 'x y' or 'x, y'.

https://docs.python.org/3/library/collections.html#collections.namedtuple

In [84]:
import timeit
from collections import namedtuple


City = namedtuple('City', 'name country population coordinates')
tokyo = City(name='Tokyo', country='JP', population=36.33, coordinates=(123.123, 345.456))
print(tokyo.name)
print(tokyo.country)
print(tokyo.population)
print(tokyo.coordinates)
print("----- print with tuples' index number ----")
print(tokyo[0])
print(tokyo[1])

Tokyo
JP
36.33
(123.123, 345.456)
----- print with tuples' index number ----
Tokyo
JP


`namedtuple()`이 메모리를 적게 사용한다지만 시간적 비용은 어떨지 궁금해서 튜플, 딕셔너리의 성능과 비교해보았다.(메모리를 왜 적게사용하는지는 아직 조사중..)<br>
timeit 모듈: https://docs.python.org/3/library/timeit.html<br>
성능 관련해서 자세한 내용 : https://stackoverflow.com/questions/2646157/what-is-the-fastest-to-access-struct-like-object-in-python


In [93]:
import timeit

setup_for_tuple = '''
tokyo = ('Tokyo', 'JP', 36.33, (123.123, 345.456))
'''
print('tuple takes ', timeit.timeit('tokyo[2]', setup=setup_for_tuple))


setup_for_namedtuple = '''
from collections import namedtuple
City = namedtuple('City', 'name country population coordinates') 
tokyo = City(name='Tokyo', country='JP', population=36.33, coordinates=(123.123, 345.456))
'''
print('namedtuple takes ', timeit.timeit('tokyo.population', setup=setup_for_namedtuple))


setup_for_dict = '''
tokyo = {
'name': 'Tokyo',
'country': 'JP',
'population': 36.33,
'coordinate': (123.123, 345.456)
}
'''
print('dict takes ', timeit.timeit("tokyo['population']", setup=setup_for_dict))

tuple takes  0.033578151000256184
namedtuple takes  0.059112099996127654
dict takes  0.03153561499493662


결론적으로 `namedtuple`은 필드를 명시하기에는 적합하지만 성능이 상대적으로 좋진 않다..